# Zindi Micro-Hydropower Energy Load Prediction

This notebook aims to predict daily energy consumption (kWh) per data user for Micro-Hydropower Plants (MHPs) in Kalam, Pakistan. We will use MHP sensor data and climate indicators to build a predictive model.

**Objective:** Forecast total daily kWh per user for one month into the future.
**Metric:** Root Mean Squared Error (RMSE).

In [2]:
# Basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math # For sqrt
import os

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Settings
SEED = 42
np.random.seed(SEED)
DATA_DIR = 'data' # Set the path to your data directory

# Ensure plots are displayed inline and set a style
%matplotlib inline
# Use a style that's likely available - adjust if needed
plt.style.use('seaborn-v0_8-darkgrid')
# If the above fails, try: plt.style.use('seaborn-darkgrid') or plt.style.use('ggplot')

print("Libraries imported and settings configured.")
print(f"Data directory set to: {DATA_DIR}")
print(f"Available plotting styles: {plt.style.available}") # Optional: see available styles

Libraries imported and settings configured.
Data directory set to: data
Available plotting styles: ['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


## 2. Load Data

Load the datasets:
1.  **MHP Data:** `Data.csv` contains the 5-minute interval sensor readings (voltage, current, kWh, etc.). We anticipate the timestamp column is named `date_time`.
2.  **Climate Data:** `Kalam Climate Data.xlsx` contains climate indicators (temperature, precipitation, etc.). This is an Excel file.
3.  **Sample Submission:** `SampleSubmission.csv` defines the required prediction format and IDs for the test set.

*Note: Reading Excel files requires the `openpyxl` library. Install it if needed (`pip install openpyxl`). Reading the MHP CSV might require `engine='python'` if the default C engine fails.*

In [ ]:
print("Loading data...")

try:
    # --- Load MHP Data (CSV) - OPTIMIZED ---
    mhp_filename = 'Data.csv'
    mhp_path = os.path.join(DATA_DIR, mhp_filename)
    print(f"Attempting to load MHP data from: {mhp_path}")

    # ---> Specify ONLY the columns needed for aggregation <---
    mhp_cols_to_load = ['date_time', 'Source', 'kwh']
    print(f"Loading only columns: {mhp_cols_to_load}")

    mhp_data_raw = pd.read_csv(
        mhp_path,
        usecols=mhp_cols_to_load,       # <--- ADDED usecols
        parse_dates=['date_time'],      # Still parse the date column
        engine='python'                 # Keep engine='python' as C engine failed before
    )
    mhp_data_raw.rename(columns={'date_time': 'timestamp', 'Source': 'user_id'}, inplace=True) # Rename here
    print(f"Successfully loaded columns from: {mhp_filename}")
    

    # --- Load Climate Data (Excel) ---
    climate_filename = 'Kalam Climate Data.xlsx'
    climate_path = os.path.join(DATA_DIR, climate_filename)
    print(f"Attempting to load climate data from: {climate_path}")
    # Use pd.read_excel for .xlsx files. Ensure openpyxl is installed.
    # Confirm the timestamp column name in the Excel file. Use that name in parse_dates.
    # Assuming it's 'timestamp' based on previous context, adjust if needed.
    climate_data_raw = pd.read_excel(climate_path, parse_dates=['Date_Time'])
    # If the Excel timestamp column was different (e.g., 'date_time'), uncomment and adjust the lines below:
    # climate_data_raw = pd.read_excel(climate_path, parse_dates=['date_time'])
    climate_data_raw.rename(columns={'Date_Time': 'timestamp'}, inplace=True)
    print(f"Successfully loaded: {climate_filename}")

    # --- Load Sample Submission (CSV) ---
    sample_sub_filename = 'SampleSubmission.csv'
    sample_sub_path = os.path.join(DATA_DIR, sample_sub_filename)
    print(f"Attempting to load sample submission from: {sample_sub_path}")
    sample_sub = pd.read_csv(sample_sub_path)
    print(f"Successfully loaded: {sample_sub_filename}")

    print("\nAll data loaded successfully.")

except FileNotFoundError as e:
    print(f"\n--- ERROR: File Not Found ---")
    print(f"{e}")
    print(f"Please ensure 'Data.csv', 'Kalam Climate Data.xlsx', and 'SampleSubmission.csv' are directly in the '{DATA_DIR}' directory.")
    raise
except ImportError as e:
     print(f"\n--- ERROR: Missing Library ---")
     print(f"{e}")
     print("Failed loading Excel file. You might need to install the required engine.")
     print("In your terminal (with venv active), run: pip install openpyxl")
     raise
except ValueError as e:
    print(f"\n--- ERROR: Value Error during loading/parsing ---")
    print(f"{e}")
    print("This might be due to an incorrect column name in 'parse_dates'. Verify column names in the source files.")
    raise
except Exception as e:
     print(f"\n--- ERROR: An unexpected error occurred during data loading ---")
     print(f"{e}")
     raise

# --- Display Info (Optional but recommended) ---
print("\n--- MHP Data Info ---")
mhp_data_raw.info()
print("\n--- Climate Data Info ---")
climate_data_raw.info()
print("\n--- Sample Submission Head ---")
print(sample_sub.head())

Loading data...
Attempting to load MHP data from: data\Data.csv
Loading only columns: ['date_time', 'Source', 'kwh']


## 3. Data Preprocessing and Aggregation

The MHP data is recorded at 5-minute intervals, but the prediction target is daily kWh per user. Climate data might also be at a finer granularity than daily.

We need to:
1.  Aggregate the MHP `kwh` readings to get the total daily sum for each `user_id`.
2.  Aggregate the climate data to daily statistics (e.g., mean temperature, total precipitation).
3.  Merge the aggregated daily MHP data with the aggregated daily climate data.
4.  Handle any missing values that might arise from the merge or exist in the original data.

In [ ]:
print("Aggregating data to daily level...")

# --- MHP Data Aggregation ---
# Ensure the timestamp column exists after loading and renaming
if 'timestamp' not in mhp_data_raw.columns:
    raise KeyError("Column 'timestamp' not found in mhp_data_raw. Check loading and renaming step.")

mhp_data = mhp_data_raw.copy()
# Extract date from the timestamp
mhp_data['date'] = mhp_data['timestamp'].dt.date
# The user identifier column seems to be 'Source' based on previous inspection, rename it to user_id for clarity
# If the user identifier is different (e.g., 'ID'), change 'Source' below
if 'Source' in mhp_data.columns:
    mhp_data.rename(columns={'Source': 'user_id'}, inplace=True)
elif 'ID' in mhp_data.columns:
     mhp_data.rename(columns={'ID': 'user_id'}, inplace=True)
else:
    raise KeyError("Could not find user identifier column ('Source' or 'ID') in mhp_data.")

# Group by the user ID and the date, then sum the kWh for that day
daily_kwh = mhp_data.groupby(['user_id', 'date'])['kwh'].sum().reset_index()
daily_kwh.rename(columns={'kwh': 'daily_kwh'}, inplace=True)
# Convert date back to datetime object for merging and feature engineering
daily_kwh['date'] = pd.to_datetime(daily_kwh['date'])
print(f"Aggregated MHP data shape: {daily_kwh.shape}")
print("Aggregated Daily kWh per User Head:")
print(daily_kwh.head())

# --- Climate Data Aggregation ---
# Ensure the timestamp column exists
if 'timestamp' not in climate_data_raw.columns:
    raise KeyError("Column 'timestamp' not found in climate_data_raw. Check loading and renaming step.")

climate_data = climate_data_raw.copy()
# Extract date from the timestamp
climate_data['date'] = climate_data['timestamp'].dt.date
# Aggregate climate features to daily level
daily_climate = climate_data.groupby('date').agg(
    temp_mean=('temperature', 'mean'),
    dew_point_mean=('dew_point', 'mean'),
    wind_speed_mean=('wind_speed', 'mean'),
    precip_sum=('precipitation', 'sum')
    # Add other aggregations if needed (min/max temp etc.)
).reset_index()
# Convert date back to datetime object
daily_climate['date'] = pd.to_datetime(daily_climate['date'])
print(f"\nAggregated climate data shape: {daily_climate.shape}")
print("Aggregated Daily Climate Head:")
print(daily_climate.head())


# --- Merge Aggregated Data ---
print("\nMerging aggregated daily MHP and climate data...")
df_train_full = pd.merge(daily_kwh, daily_climate, on='date', how='left')
print(f"Initial merged shape: {df_train_full.shape}")

# --- Handle Missing Values (Simple Strategy) ---
# Check NaNs introduced by merge or existing in climate data
print(f"\nNaNs before fill:\n{df_train_full.isnull().sum()}")
# Use forward fill first to propagate last known values, then back fill for any remaining at the start
df_train_full.sort_values(by=['user_id', 'date'], inplace=True) # Sort before filling
df_train_full.fillna(method='ffill', inplace=True)
df_train_full.fillna(method='bfill', inplace=True)
print(f"\nNaNs after fill: {df_train_full.isnull().sum().sum()}") # Should ideally be 0

print("\n--- Merged & Cleaned Training Data Head ---")
print(df_train_full.head())

## 4. Feature Engineering

Create features based on the date and potentially other aspects of the data. For this baseline, we will focus on date-based features. More advanced features (lags, rolling windows, user-specific stats) can be added later for improvement.

In [ ]:
print("Creating date-based features...")

def create_date_features(df, date_col='date'):
    """Creates time series features from a date column."""
    df = df.copy() # Avoid SettingWithCopyWarning
    df[date_col] = pd.to_datetime(df[date_col]) # Ensure it's datetime
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day'] = df[date_col].dt.day
    df['dayofweek'] = df[date_col].dt.dayofweek # Monday=0, Sunday=6
    df['dayofyear'] = df[date_col].dt.dayofyear
    # Use .dt.isocalendar().week for ISO standard week number
    df['weekofyear'] = df[date_col].dt.isocalendar().week.astype(int)
    df['quarter'] = df[date_col].dt.quarter
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    # Example: Add sine/cosine features for month (captures cyclicity)
    # df['month_sin'] = np.sin(2 * np.pi * df['month']/12.0)
    # df['month_cos'] = np.cos(2 * np.pi * df['month']/12.0)
    return df

# Apply feature creation function to the merged data
df_train_full = create_date_features(df_train_full, 'date')

# --- Define Features and Target ---
# List of columns to be used as input features for the model
features = [
    # Date features
    'year', 'month', 'day', 'dayofweek', 'dayofyear', 'weekofyear', 'quarter', 'is_weekend',
    # Climate features
    'temp_mean', 'dew_point_mean', 'wind_speed_mean', 'precip_sum'
    # Add more features here later:
    # e.g., lag features, rolling window features, user id encodings
]
# The column we want to predict
target = 'daily_kwh'

# Make sure all feature columns are numeric for the model (important for some models)
print("\nChecking feature data types...")
for col in features:
    if col not in df_train_full.columns:
        raise ValueError(f"Feature '{col}' not found in dataframe columns: {df_train_full.columns}")
    if not pd.api.types.is_numeric_dtype(df_train_full[col]):
        print(f"Warning: Feature '{col}' is not numeric ({df_train_full[col].dtype}). Attempting conversion.")
        df_train_full[col] = pd.to_numeric(df_train_full[col], errors='coerce')
        # Handle potential NaNs created by coerce if conversion fails
        if df_train_full[col].isnull().any():
             print(f"NaNs created in {col} after conversion, filling with 0")
             df_train_full[col].fillna(0, inplace=True) # Or use mean/median

print("Features defined and checked.")
print(f"List of features ({len(features)}): {features}")

print("\n--- Training Data with Features Head ---")
print(df_train_full.head())

## 5. Train/Validation Split (Time-Based)

For time series forecasting, it's crucial to validate the model on data that comes *after* the training data. We will split the data chronologically, using the most recent period for validation. Shuffling should **not** be used.

In [ ]:
print("Splitting data into train and validation sets based on time...")

# Ensure data is sorted chronologically overall (important for time-based split)
# Sorting by user then date is already done, but an overall sort helps define the cutoff
df_train_full = df_train_full.sort_values(by='date')

# Define validation period (e.g., last 30 days of available data)
# Find the overall max date in the dataset
max_date = df_train_full['date'].max()
# Calculate the start date for the validation set
# Ensure there are enough days for a meaningful validation set
validation_days = 30
validation_start_date = max_date - pd.Timedelta(days=validation_days - 1) # -1 because we include the start date

print(f"Overall data range: {df_train_full['date'].min()} to {max_date}")
print(f"Using data from {validation_start_date} onwards for validation ({validation_days} days).")

# Split the data
train_data = df_train_full[df_train_full['date'] < validation_start_date].copy()
val_data = df_train_full[df_train_full['date'] >= validation_start_date].copy()

# Ensure there's data in both sets
if train_data.empty or val_data.empty:
     raise ValueError("Train or validation set is empty. Check data range and validation_days.")

print(f"\nTraining data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Training data period: {train_data['date'].min().date()} to {train_data['date'].max().date()}")
print(f"Validation data period: {val_data['date'].min().date()} to {val_data['date'].max().date()}")

# Separate features (X) and target (y) for train and validation sets
X_train = train_data[features]
y_train = train_data[target]
X_val = val_data[features]
y_val = val_data[target]

print("\nTrain/Validation split complete.")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

## 6. Model Training (LightGBM Baseline)

We will use LightGBM, a gradient boosting framework known for its speed and efficiency, to train a baseline model. We'll use early stopping based on the validation set performance (RMSE) to prevent overfitting and find a reasonable number of boosting rounds.

In [ ]:
print("Training LightGBM model...")

# Define LightGBM parameters
lgb_params = {
    'objective': 'regression_l1', # Use 'regression_l1' (MAE) for robustness to outliers, or 'regression_l2' (RMSE)
    'metric': 'rmse',             # Evaluation metric used for early stopping
    'n_estimators': 2000,         # Max number of trees; early stopping will find the optimal number
    'learning_rate': 0.03,        # Step size shrinkage
    'feature_fraction': 0.8,      # Fraction of features to consider per tree
    'bagging_fraction': 0.8,      # Fraction of data to sample per tree (requires bagging_freq > 0)
    'bagging_freq': 1,            # Perform bagging on every iteration
    'lambda_l1': 0.1,             # L1 regularization
    'lambda_l2': 0.1,             # L2 regularization
    'num_leaves': 31,             # Max number of leaves in one tree (controls complexity)
    'verbose': -1,                # Suppress verbose logging during training
    'n_jobs': -1,                 # Use all available CPU cores
    'seed': SEED,                 # Random seed for reproducibility
    'boosting_type': 'gbdt',      # Standard gradient boosted decision trees
}

# Initialize the model
model = lgb.LGBMRegressor(**lgb_params)

# Train the model with early stopping
print("Starting model fitting...")
model.fit(X_train, y_train,
          eval_set=[(X_val, y_val)],          # Data to evaluate on during training
          eval_metric='rmse',                # Metric for evaluation
          callbacks=[lgb.early_stopping(stopping_rounds=100, # Stop if metric doesn't improve for 100 rounds
                                        verbose=100)])      # Print progress every 100 rounds

print("\nModel training complete.")
print(f"Best iteration found by early stopping: {model.best_iteration_}")

## 7. Local Validation (Calculate RMSE)

Evaluate the trained model's performance on the unseen validation set. This gives us an estimate of how well the model might perform on the actual test data on Zindi. We will calculate the Root Mean Squared Error (RMSE).

In [ ]:
print("Evaluating model on the validation set...")

# Predict on the validation set features
val_preds = model.predict(X_val)

# Optional: Ensure no negative predictions (kWh cannot be negative)
val_preds_non_negative = np.maximum(0, val_preds)
negative_preds_count = np.sum(val_preds < 0)
if negative_preds_count > 0:
    print(f"Warning: {negative_preds_count} negative predictions were clipped to 0.")

# --- Calculate RMSE using the non-negative predictions ---
rmse = math.sqrt(mean_squared_error(y_val, val_preds_non_negative))
print(f"\nValidation RMSE: {rmse:.4f}")

# --- Feature Importance Plot ---
# Plotting helps understand which features the model found most useful
print("\nPlotting feature importance...")
try:
    lgb.plot_importance(model, figsize=(10, max(6, len(features)//2)), max_num_features=len(features))
    plt.title(f'LightGBM Feature Importance (Validation RMSE: {rmse:.4f})')
    plt.tight_layout()
    plt.show()
except Exception as plot_err:
    print(f"Could not plot feature importance: {plot_err}")

# Optional: Analyze Residuals
# residuals = y_val - val_preds_non_negative
# plt.figure(figsize=(10, 6))
# sns.histplot(residuals, bins=50, kde=True)
# plt.title('Distribution of Residuals (Actual - Predicted)')
# plt.xlabel('Error (kWh)')
# plt.show()

# plt.figure(figsize=(10, 6))
# plt.scatter(val_preds_non_negative, residuals, alpha=0.5)
# plt.title('Residuals vs. Predicted Values')
# plt.xlabel('Predicted kWh')
# plt.ylabel('Residual (Actual - Predicted)')
# plt.axhline(0, color='red', linestyle='--')
# plt.show()

## 8. Prepare Test Data and Generate Predictions

Now we prepare the actual test dataset based on the `SampleSubmission.csv` file. This involves:
1.  Extracting the required future `date` and `user_id` from the `ID` column in the sample submission.
2.  Merging the relevant aggregated daily `climate` data for those future dates.
3.  Creating the same date-based `features` that the model was trained on.
4.  Handling any potential missing climate data for future dates (e.g., by forward filling the last known values).
5.  Using the trained `model` to predict `daily_kwh` for the test set.

In [ ]:
print("Preparing test data for final prediction...")

# --- Create Test Set Structure from Sample Submission ---
if 'ID' not in sample_sub.columns:
    raise KeyError("Column 'ID' not found in SampleSubmission.csv.")

test_df = sample_sub[['ID']].copy()
# Extract date and user_id from the submission ID string
try:
    test_df['date_str'] = test_df['ID'].str.split('_').str[0]
    test_df['date'] = pd.to_datetime(test_df['date_str'], errors='coerce')
    # Reconstruct user_id (everything after the date and the first underscore)
    test_df['user_id'] = test_df['ID'].str.split('_', n=1).str[1]
except Exception as e:
    print(f"Error parsing ID column in sample submission: {e}")
    raise

# Drop rows where date parsing failed, if any
invalid_dates = test_df['date'].isnull().sum()
if invalid_dates > 0:
    print(f"Warning: Removed {invalid_dates} rows from test set due to invalid date format in ID.")
    test_df.dropna(subset=['date'], inplace=True)

print(f"Test set initial structure created. Shape: {test_df.shape}")
print("Test DataFrame Head (with parsed date/user):")
print(test_df.head())

# --- Merge Future Climate Data ---
# Use the aggregated daily climate data prepared earlier
test_df = pd.merge(test_df, daily_climate, on='date', how='left')
print(f"Test set shape after merging climate data: {test_df.shape}")

# --- Handle Missing Climate Data for Future Dates ---
# Important: Future dates might not have climate data in our historical set.
# Sort by date first to ensure correct forward filling
test_df.sort_values(by='date', inplace=True)
# Forward fill using the last known climate values
test_df.fillna(method='ffill', inplace=True)

# Check if any NaNs remain (e.g., if test dates are *before* the first climate date)
remaining_nans = test_df[daily_climate.columns.difference(['date'])].isnull().sum()
if remaining_nans.sum() > 0:
     print(f"\nWarning: NaNs remain in test climate data after forward fill:")
     print(remaining_nans[remaining_nans > 0])
     print("This might happen if test dates are before the start of climate data.")
     print("Filling remaining NaNs with 0 - consider a more robust strategy (e.g., overall mean/median) if this occurs.")
     test_df.fillna(0, inplace=True) # Simple strategy for remaining NaNs

# --- Create Features for Test Set ---
print("\nCreating features for the test set...")
test_df = create_date_features(test_df, 'date')

# --- Align Test Columns with Training Features ---
# Ensure test set has exactly the same feature columns as the training set, in the same order
try:
    X_test = test_df[features]
except KeyError as e:
    print(f"Error aligning test features: Missing column {e}")
    print(f"Columns available in test_df: {test_df.columns.tolist()}")
    print(f"Features expected by model: {features}")
    raise

# Check for NaNs in the final test features (should not happen after fillna steps)
if X_test.isnull().any().any():
    print("\n--- FATAL ERROR: NaNs found in final test features before prediction! ---")
    print(X_test.isnull().sum())
    raise ValueError("NaNs detected in test features. Check data preparation and fillna steps.")
else:
    print("\nTest features successfully prepared and checked for NaNs.")

print(f"Final test data shape for prediction (X_test): {X_test.shape}")
print("Test Data Head (Ready for Prediction):")
print(X_test.head())

# --- Predict on Test Set ---
print("\nPredicting on the prepared test set...")
test_predictions = model.predict(X_test)

# Ensure no negative predictions
test_predictions_non_negative = np.maximum(0, test_predictions)
test_neg_count = np.sum(test_predictions < 0)
if test_neg_count > 0:
    print(f"Note: {test_neg_count} negative predictions in test set were clipped to 0.")

print("Prediction on test set complete.")

## 9. Generate Submission File

Create the final submission file in the format required by Zindi: a CSV file with two columns, `ID` and `kwh`. The `ID` column must match the `SampleSubmission.csv`.

In [ ]:
print("Generating submission file...")

# Create the submission DataFrame
submission_df = pd.DataFrame({'ID': test_df['ID'], 'kwh': test_predictions_non_negative})

# Define the submission filename
submission_filename = 'submission_baseline_lgb_v1.csv' # Increment version number as you improve

# Save the submission file
submission_df.to_csv(submission_filename, index=False)

print("\n--- Submission File Head ---")
print(submission_df.head())
print(f"\nSubmission file saved successfully as: {submission_filename}")
print(f"File shape: {submission_df.shape}")

# Optional: Check if the number of rows matches the sample submission
if len(submission_df) == len(sample_sub):
    print("Number of rows matches SampleSubmission.csv.")
else:
    print(f"Warning: Row count mismatch! Submission has {len(submission_df)} rows, SampleSub has {len(sample_sub)} rows.")

--- End of Baseline Notebook ---

Next steps:
- Submit the generated CSV to Zindi.
- Analyze the results (local RMSE vs Zindi score).
- Improve the model by:
    - Adding more features (lags, rolling windows, user features).
    - Tuning hyperparameters (e.g., using Optuna).
    - Trying different models (XGBoost, CatBoost).
    - Implementing more robust validation (Time Series Cross-Validation).
    - Ensembling models.